# Setup IAM for Kinesis

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)


# Create Kinesis Role

In [ ]:
iam_kinesis_role_name = 'DSOAWS_Kinesis'

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesis.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "firehose.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesisanalytics.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }            
  ]
} 

In [ ]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_kinesis = iam.create_role(
        RoleName=iam_kinesis_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Kinesis Role'
    )
    print('Role succesfully created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_kinesis = iam.get_role(RoleName=iam_kinesis_role_name)
        print('Role already exists. That is OK.')
    else:
        print('Unexpected error: %s' % e)
        
time.sleep(10)

In [ ]:
iam_role_kinesis_name = iam_role_kinesis['Role']['RoleName']
print('Role Name: {}'.format(iam_role_kinesis_name))

In [ ]:
iam_role_kinesis_arn = iam_role_kinesis['Role']['Arn']
print('Role ARN: {}'.format(iam_role_kinesis_arn))

In [ ]:
account_id = sts.get_caller_identity()['Account']

# Specify Stream Name

In [ ]:
stream_name = 'dsoaws-kinesis-data-stream'

# Specify Firehose Name

In [ ]:
firehose_name = 'dsoaws-kinesis-data-firehose'

# Specify Lambda Function Name

In [ ]:
lambda_fn_name = 'DeliverKinesisAnalyticsToCloudWatch'

# Create Policy

In [ ]:
kinesis_policy_doc = {
    
    "Version": "2012-10-17",
    "Statement": [
        {      
            "Effect": "Allow",      
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject"
            ],      
            "Resource": [        
                "arn:aws:s3:::{}/kinesis-data-firehose".format(bucket),
                "arn:aws:s3:::{}/kinesis-data-firehose/*".format(bucket)
            ]    
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:{}:{}:log-group:/*".format(region, account_id)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesis:Get*",
                "kinesis:DescribeStream",
                "kinesis:Put*",
                "kinesis:List*",
            ],
            "Resource": [
                "arn:aws:kinesis:{}:{}:stream/{}".format(region, account_id, stream_name)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "firehose:*",
            ],
            "Resource": [
                "arn:aws:firehose:{}:{}:deliverystream/{}".format(region, account_id, firehose_name)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesisanalytics:*",
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Resource": "arn:aws:lambda:{}:{}:function:{}:$LATEST".format(region, account_id, lambda_fn_name)
        },
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/service-role/kinesis-analytics*"
        }
    ]
}

print(json.dumps(kinesis_policy_doc, indent=4, sort_keys=True, default=str))

# Update Policy

In [ ]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_kinesis_name,
    PolicyName='DSOAWS_KinesisPolicy',
    PolicyDocument=json.dumps(kinesis_policy_doc)
)

time.sleep(10)

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

In [ ]:
%store stream_name

In [ ]:
%store firehose_name

In [ ]:
%store iam_kinesis_role_name

In [ ]:
%store iam_role_kinesis_arn

In [ ]:
%store lambda_fn_name

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();